In [2]:
import pandas as pd

# Import only first 1000 lines for testing
data = pd.read_csv('./data/sensor_readings_timeseries.csv', nrows=1000)

# Prep input data


     Humidity Metadata.Location.coordinates Metadata.Location.type  \
0       95.12          [10.252,53.502,38.9]                  Point   
1       92.67          [10.252,53.502,38.9]                  Point   
2       93.64          [10.252,53.502,38.9]                  Point   
3       92.76          [10.252,53.502,38.9]                  Point   
4       91.67          [10.252,53.502,38.9]                  Point   
..        ...                           ...                    ...   
995     72.96              [10.4,53.226,37]                  Point   
996     73.00              [10.4,53.226,37]                  Point   
997     72.95              [10.4,53.226,37]                  Point   
998     72.88              [10.4,53.226,37]                  Point   
999     72.92              [10.4,53.226,37]                  Point   

     Metadata.NetatmoSensorId Metadata.Provider  \
0                         NaN  sensor.community   
1                         NaN  sensor.community   
2     

In [3]:
from sklearn import linear_model

reg = linear_model.Lasso(alpha=0.1)
reg.fit([[0, 0], [1, 1]], [0, 1])
print(reg.coef_)


KeyboardInterrupt: 